# How to connect to Azure OpenAI service via PromptSail and Langchain SDK


First you need to get or create an Azure OpenAI resource. 
You can do it here: https://portal.azure.com/#view/Microsoft_Azure_ProjectOxford/CognitiveServicesHub/~/OpenAI

Click on the chosen resource and than the "Develop" button in the middle of the screen.

Copy the API key and endpoint and paste it into `.env` file in the `examples` folder. 

Install all the necessary packages from [examples/pyproject.toml](pyproject.toml) by running the following command:

```bash 
cd prompt_sail/examples
poetry install
```


## Test the connection without PromptSail

Import the OpenAI Python SDK and load your API key from the environment.

In [1]:
import os
from dotenv import load_dotenv, dotenv_values
from rich import print  


config = dotenv_values(".env")


azure_oai_key = config["AZURE_OPENAI_API_KEY"]
api_base_url = config["AZURE_OPENAI_ENDPOINT"]

deployment_name = config["AZURE_LLM_DEPLOYMENT_NAME"]
api_version = config["AZURE_OPENAI_API_VERSION"]

print(
    f"Azure OpenAI api key={azure_oai_key[0:3]}...{azure_oai_key[-5:]}"
)
print(
    f"Azure OpenAI api endpoint={api_base_url[0:17]}..."
)
print(f"Azure OpenAI deployment name={deployment_name[0:7]}...")
print(f"Azure OpenAI api version={api_version}")

Azure OpenAI api key=9b2...d67b3

Azure OpenAI api endpoint=https://openai-pr...

Azure OpenAI deployment name=gpt-35T...

Azure OpenAI api version=2023-07-01-preview

Test the direct connection to Azure OpenAI and Langchain SDK.


In [2]:
from langchain_openai  import AzureChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage, ChatMessage

from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate


message = [
    SystemMessage(
        content="You are a helpful assistant that help rewirte an jira ticket."
    ),
    HumanMessage(
        content="Give meaningful title to this bug, RuntimeError: CUDA out of memory. Tried to allocate X MiB (GPU X; X GiB total capacity; X GiB already allocated; X MiB free; X cached)"
    ),
]

message_followup = [
    SystemMessage(
        content="You are a helpful assistant that help rewirte an jira ticket."
    ),
    HumanMessage(
        content="Give meaningful title to this bug, RuntimeError: CUDA out of memory. Tried to allocate X MiB (GPU X; X GiB total capacity; X GiB already allocated; X MiB free; X cached)"
    ),
    AIMessage(
        content="RuntimeError: CUDA out of memory. Tried to allocate X MiB (GPU X; X GiB total capacity; X GiB already allocated; X MiB free; X cached)"
    ),
    HumanMessage(
        content="The error message is too technical, could you make it more user friendly and poetic?"
    ),
]

In [3]:
chat = AzureChatOpenAI(
    openai_api_key=azure_oai_key,
    deployment_name=deployment_name,
    api_version=api_version,
    azure_endpoint=api_base_url,
    #base_url=api_base_url,
    #model=deployment_name,
)

In [4]:
resp = chat.invoke(message)
print(resp) 

AIMessage(
    content='Bug: RuntimeError: CUDA out of memory while allocating X MiB (GPU X; X GiB total capacity; X GiB 
already allocated; X MiB free; X cached)',
    response_metadata={
        'token_usage': {'completion_tokens': 37, 'prompt_tokens': 70, 'total_tokens': 107},
        'model_name': 'gpt-35-turbo',
        'system_fingerprint': None,
        'prompt_filter_results': [
            {
                'prompt_index': 0,
                'content_filter_results': {
                    'hate': {'filtered': False, 'severity': 'safe'},
                    'self_harm': {'filtered': False, 'severity': 'safe'},
                    'sexual': {'filtered': False, 'severity': 'safe'},
                    'violence': {'filtered': False, 'severity': 'safe'}
                }
            }
        ],
        'finish_reason': 'stop',
        'logprobs': None,
        'content_filter_results': {
            'hate': {'filtered': False, 'severity': 'safe'},
            'self_harm': {'filtered': False, 'severity': 'safe'},
            'sexual': {'filtered': False, 'severity': 'safe'},
            'violence': {'filtered': False, 'severity': 'safe'}
        }
    },
    id='run-05f04842-cf2d-44ad-a2a2-0f8d70acb398-0',
    usage_metadata={'input_tokens': 70, 'output_tokens': 37, 'total_tokens': 107}
)

## Create a request to the AzureOpenAI via PromptSail proxy


* Go to demo [Try PromptSail](https://try-promptsail.azurewebsites.net/) and create a new project or use an existing one.
* [Run the PromptSail docker images](https://promptsail.com/docs/quick-start-guide/#pull-and-run-the-docker-images-from-ghcr) and go to UI at http://localhost/.
We will have to setup a project and add ai provider. 

Create new project with you `project_slug`or edit existing one.

Add your own Azure OpenAI provider by editing the project settings, this will map the Azure OpenAI endpoint to new proxy prompt sail URL. 

Set the `api base url` to your Azure OpenAI endpoint like
 
'https://**azure_openai_resource**.openai.azure.com/'
 
 and add meaningfull `deployment name`.


In mongo it will create new entry in `ai_providers` array, similar to this one

```bash
{
     ai_providers: [
        {
            deployment_name: 'azure US deployment',
            slug: 'azure-us-deployment',
            api_base: 'https://[azure_openai_resource].openai.azure.com/',
            description: '',
            provider_name: 'Azure OpenAI'
        }
    ],
}
```

In this case we will use the default `Client campaign` settings:
* with project_slug -> 'client-campaign' 
* deployment_name -> 'azure-us-deployment'
resulting in promptsail proxy url like this: 

**"http://localhost:8000/client-campaign/azure-us-deployment/"**



In [26]:
ps_api_base = "http://localhost:8000/client-campaign/azure-prompt-sail-eus2/"

chat = AzureChatOpenAI(
    openai_api_key=azure_oai_key,
    deployment_name=deployment_name,
    api_version=api_version,
    azure_endpoint=ps_api_base,

)

In [27]:
resp = chat.invoke(message)
print(resp)

### Azure OpenAI costs calculation and model versions


The Azure OpenAI responses do not include the version of the model used to generate the response, request contains the `deployment_name` and response returns just model famili eg, `gpt-3.5-turbo` and not `gpt-3.5-turbo-1106`, this makes impossible to track and calculate costs accurately. To address this issue, you can pass the model_version parameter to the AzureChatOpenAI class, which will append the version to the model name in the output. This allows for easy differentiation between different versions of the model.


Ensure that your deployment supports the chosen model version, as different regions support various models and versions. You can verify the supported models and versions in each region by following this
[Azure OpenAI Model summary table and region availability](https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/models#model-summary-table-and-region-availability)

In [40]:
ps_api_base_model_version_tag = "http://localhost:8000/client-campaign/azure-prompt-sail-eus2/?tags=examples,langchain-v0.0.354&target_path="

#ps_api_base_model_version_tag = "https://try-promptsail.azurewebsites.net/api/models-playground/azure-eastus/?ai_model_version=gpt-4o-2024-05-13&target_path="

chat_0613 = AzureChatOpenAI(
    openai_api_key=azure_oai_key,
    deployment_name=deployment_name,
    api_version=api_version,
    azure_endpoint=ps_api_base_model_version_tag,
    #model_version="0613",
)

In [41]:
chat_0613(message_followup)